In [ ]:
#Dieser Code beschäftigt sich damit, die ERA5-Daten umzurechnen in Durchschnittswerte und der Berechnung der Windrichtungsdateien

In [2]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import shapefile
from shapely.geometry import Point, Polygon, shape
import shapely
import math
from numpy import genfromtxt
import time
import netCDF4 as nc
from datetime import datetime

In [3]:
#Diese Definition korrigiert die Verengung der Längengrade nach Norden/Süden hin

def durschn(array,Gewichtungsmatrix):   #Berechnet den Durchschnitt für die Regionen aus den Koordinatendaten, achtet dabei auf Fehlwerte
    bfakt=sum(sum(Gewichtungsmatrix))/(len(array)**2-np.count_nonzero(Gewichtungsmatrix == 0))
    Schnitt=sum(sum(array))/((len(array)**2-np.count_nonzero(array == 0))*bfakt)
    if float(Schnitt) > 0:
        al=1 #Platzhalter, bringt nix außer dass ich else schreiben kann
    else:
        array1=np.nan_to_num(array)
        bfakt=sum(sum(Gewichtungsmatrix))/(len(array1)**2-np.count_nonzero(Gewichtungsmatrix == 0))
        Schnitt=sum(sum(array1))/((len(array1)**2-np.count_nonzero(array1 == 0))*bfakt)                     
    return Schnitt

In [ ]:
#Hilfsmittel, weil ich erst CDO 1.9.9 benutzt habe, was nicht funktioniert hat
lu=[]
for i in range (1,43):
    gdf1=xr.open_dataset('/pd/home/weber-j/Zwischendateien/'+str(i)+'.nc')
    globals()['gd'+str(i)]=gdf1
gdfs1=xr.merge([gd1,gd2,gd3,gd4,gd5,gd6,gd7,gd8,gd9,gd10])
gdfs2=xr.merge([gd11,gd12,gd13,gd14,gd15,gd16,gd17,gd18,gd19,gd20])
gdfs3=xr.merge([gd21,gd22,gd23,gd24,gd25,gd26,gd27,gd28,gd29,gd30,gd31])
gdfs4=xr.merge([gd32,gd33,gd34,gd35,gd36,gd37,gd38,gd39,gd40,gd41,gd42])
gddf=xr.merge([gdfs1,gdfs2,gdfs3,gdfs4])
xr.Dataset.to_netcdf(gddf,'/pd/home/weber-j/Zwischendateien/Grunddata_sst.nc')

In [ ]:
gdf=xr.open_dataset('/Users/weber-j/Documents/u750_all.nc')

In [ ]:
gdf.time

In [ ]:
gdf=xr.open_dataset('/Users/weber-j/Documents/u750_all.nc')
if len(gdf.time) > 14750:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(688,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
start=time.time()
print(gdf.time[0])

In [3]:
#Erschaffen der Durchschnittsdateien, habe ich jeweils die Variablen nacheinander eingesetzt
gdf=xr.open_dataset('/Users/weber-j/Documents/wr900_all.nc')
if len(gdf.time) > 14750 and len(gdf.time) < 15000:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(688,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
start=time.time()
for Jahr in range (0,39):
    t=Jahr*365+59
    sch=int((Jahr)/4)   #Schaltjahr
    anfang=t+sch
    ende=anfang+31+30+31
    for k in range (anfang,ende):
        if k == 59:
            gd=gdf.u[k]
        else:
            gd=gd+gdf.u[k]
    end=time.time()
    print('Jahr:',Jahr+1980,'    Zeit:',end-start)
gdf_neu=gd/(39*92)

#Schreiben der neuen Datei
fna = '/Users/weber-j/Documents/Mean_wr900.nc' 
ds1 = nc.Dataset(fna, 'w', format='NETCDF4')
lat = ds1.createDimension('lat', 100)
lon = ds1.createDimension('lon', 88)
lt = ds1.createVariable('lat', 'f8', ('lat',))
ln = ds1.createVariable('lon', 'f8', ('lon',))
wr = ds1.createVariable('wr', 'f8', ('lat', 'lon',))

lt[:] = np.arange(14.25, -10.75, -0.25)
ln[:] = np.arange(30, 52, 0.25)

wr[:, :] = gdf_neu

wr.long_name = 'Mean wind direction at 900 hPa niveau'
wr.unit = 'Degrees'
now = datetime.now()   #Für das aktuelle Datum
now1=int(now.strftime("%Y"))     #Fürs Erstellungsdatum
now2=int(now.strftime("%m"))
now3=int(now.strftime("%d"))
now4=int(now.strftime("%H"))
now5=int(now.strftime("%M"))
now6=int(now.strftime("%S"))
now2='{:02}'.format(now2)
now3='{:02}'.format(now3)
now4='{:02}'.format(now4)
now5='{:02}'.format(now5)
now6='{:02}'.format(now6)

#Globale Attribute schreiben
ds1.Title = 'ERA5 GHA Mean'
ds1.Institution = 'Karlsruhe Institute of Technology - Institute of Meteorology and Climate Research'
ds1.Source = 'ECMWF ERA5'
ds1.Comment = ''
ds1.History = (str(now1)+'-'+str(now2)+'-'+str(now3)+' '+str(now4)+':'+str(now5)+':'+str(now6)+' : File created.')
ds1.Contact_Person = 'Jan Niklas Weber (jan.weber9@kit.edu)'
ds1.Author = 'Jan Niklas Weber (jan.weber9@kit.edu)'
ds1.License = 'For non-commercial use only'
ds1.close()
print('finished')

Jahr: 1980     Zeit: 0.22320199012756348
Jahr: 1981     Zeit: 0.4026830196380615
Jahr: 1982     Zeit: 0.6282308101654053
Jahr: 1983     Zeit: 0.7961390018463135
Jahr: 1984     Zeit: 0.9621548652648926
Jahr: 1985     Zeit: 1.107917070388794
Jahr: 1986     Zeit: 1.2773077487945557
Jahr: 1987     Zeit: 1.4517607688903809
Jahr: 1988     Zeit: 1.6498308181762695
Jahr: 1989     Zeit: 1.8500800132751465
Jahr: 1990     Zeit: 2.055445909500122
Jahr: 1991     Zeit: 2.2551980018615723
Jahr: 1992     Zeit: 2.4564640522003174
Jahr: 1993     Zeit: 2.6099770069122314
Jahr: 1994     Zeit: 2.798084020614624
Jahr: 1995     Zeit: 2.9581470489501953
Jahr: 1996     Zeit: 3.138016939163208
Jahr: 1997     Zeit: 3.295348882675171
Jahr: 1998     Zeit: 3.48396897315979
Jahr: 1999     Zeit: 3.6309990882873535
Jahr: 2000     Zeit: 3.855835199356079
Jahr: 2001     Zeit: 4.009766101837158
Jahr: 2002     Zeit: 4.214481830596924
Jahr: 2003     Zeit: 4.427006006240845
Jahr: 2004     Zeit: 4.641072750091553
Jahr: 2005 

In [ ]:
#Checken der Ergebnisse
xr.open_dataset('/Users/weber-j/Documents/Mean_wr900.nc')

In [ ]:
#Windrichtungsberechnung
wr=180+180/np.pi*np.arctan2(uw.u,vw.v)

In [ ]:
wr

In [ ]:
alt=xr.open_dataset('/Users/weber-j/Documents/u750_all_Test.nc')
alt

In [ ]:
alt0=alt*0

In [ ]:
alt1=alt0+1
alt1.u[0,0]*wr[0,0]

In [ ]:
gdf.u[tim,lev]=(gdf.u[tim,lev]*0+1)*(wr[tim,lev])

In [ ]:
#Erstellen von Windrichtung-Datensets
vw=xr.open_dataset('/Users/weber-j/Documents/v900_all.nc')
uw=xr.open_dataset('/Users/weber-j/Documents/u900_all.nc')
wr=180+180/np.pi*np.arctan2(uw.u,vw.v)
gdf=xr.open_dataset('/Users/weber-j/Documents/u750_all_Test.nc')
start=time.time()
for tim in range (0,15341):
    for lev in range (0,1):
        gdf.u[tim,lev]=(gdf.u[tim,lev]*0+1)*(wr[tim,lev])
    end=time.time()
    if float(tim/1000)==int(tim/1000):
        print('Tag:',tim,'   Zeit:',end-start)
print('The bitter end')
xr.Dataset.to_netcdf(gdf,'/Users/weber-j/Documents/wr900_all.nc')

In [31]:
#Erstellen einer Gewichtungsmatrix für die komplette Area
r = shapefile.Reader('/Users/weber-j/Documents/Masterarbeit/GHA_Shapedateien/GHA.shp')
shapes = r.shapes()

gd=xr.open_dataset('/Users/weber-j/Documents/ws10_all.nc')
lati=[]
for r in range (0,len(gd.latitude)):
    lati.append(float(gd.latitude[r]))
Deucor=np.cos(np.deg2rad(lati))*6371*2*math.pi/(np.cos(np.deg2rad(0))*6371*2*math.pi)  #Das wird für die Breitenkorrektur gebraucht
Deucor

array([0.96923091, 0.97029573, 0.97134207, 0.97236992, 0.97337926,
       0.97437006, 0.97534232, 0.97629601, 0.97723111, 0.9781476 ,
       0.97904547, 0.9799247 , 0.98078528, 0.98162718, 0.9824504 ,
       0.98325491, 0.9840407 , 0.98480775, 0.98555606, 0.9862856 ,
       0.98699637, 0.98768834, 0.98836151, 0.98901586, 0.98965139,
       0.99026807, 0.9908659 , 0.99144486, 0.99200495, 0.99254615,
       0.99306846, 0.99357186, 0.99405634, 0.9945219 , 0.99496852,
       0.9953962 , 0.99580493, 0.9961947 , 0.9965655 , 0.99691733,
       0.99725019, 0.99756405, 0.99785892, 0.9981348 , 0.99839167,
       0.99862953, 0.99884839, 0.99904822, 0.99922904, 0.99939083,
       0.99953359, 0.99965732, 0.99976203, 0.9998477 , 0.99991433,
       0.99996192, 0.99999048, 1.        , 0.99999048, 0.99996192,
       0.99991433, 0.9998477 , 0.99976203, 0.99965732, 0.99953359,
       0.99939083, 0.99922904, 0.99904822, 0.99884839, 0.99862953,
       0.99839167, 0.9981348 , 0.99785892, 0.99756405, 0.99725

In [32]:
li=[]
lii=[]
for b in range (0,100):
    for a in range (0,88):
        li.append(Deucor[b])
    lii.append(li)
    li=[]
lii=np.asarray(lii)

In [ ]:
#Saisonalen Durchschnitt berechnen (für monatlichen Schnitt die Tage anpassen jeweils)

#Saisonal pro Land
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']

filename='/Users/weber-j/Documents/tp_all.nc'

gdf=xr.open_dataset(filename)
if filename == '/Users/weber-j/Documents/tp_all.nc':   #Regendaten sind kaputt (1979 doppelt drin), geht nur so
    gdf=gdf.isel(lon=slice(0,88),time=slice(687,15000))
    tp_Zusatz=0
else:
    gdf=gdf.isel(lon=slice(0,88),time=slice(365,15000))
    tp_Zusatz=1
for Jahr in range (0,39):
    t=Jahr*365+59
    sch=int((Jahr)/4)   #Schaltjahr
    anfang=t+sch+tp_Zusatz
    ende=anfang+31+30+31
    gd=durschn(gdf.var1[anfang]*lii,lii)
    for k in range (anfang+1,ende):
        gd=gd+durschn(gdf.var1[k]*lii,lii)
    print(float(gd)/92)
print('Until the end of the line')

In [ ]:
sst=xr.open_dataset('/Users/weber-j/Documents/sst_all.nc')
sst.lon

In [ ]:
gd=gd+durschn(gdf.v[k,0]*lii,lii)

In [ ]:
#Analyse eines einzelnen Punktes (Max-Niederschlag)
gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')
gdf=gdf.tp[0:1000000,52,22]
ju=0
for Jahr in range (1,40): #(0,40)
    t=Jahr*365+59
    sch=int((Jahr+3)/4)   #Schaltjahr
    anfang=t+sch+61
    ende=anfang+31#+31+30+31
    gd=gdf[anfang]
    for k in range (anfang+1,ende):
        gd=gd+gdf[k]
    print(float(gd)/31)
    ju=ju+float(gd)/31
ju=ju/39

In [ ]:
ju

In [ ]:
for j in range (7455,7455+92):
    print(float(gdf[j]))

In [ ]:
#Saisonal pro Land
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']

filename='/Users/weber-j/Documents/tp_all.nc'

gdf=xr.open_dataset(filename)
if filename == '/Users/weber-j/Documents/tp_all.nc':   #Regendaten sind kaputt (1979 doppelt drin), geht nur so
    gdf=gdf.isel(longitude=slice(0,88),time=slice(688,15000))
    tp_Zusatz=0
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
    tp_Zusatz=1
for ma in range (3,4): #(0,12)
    print(liste[ma],'   ########################################################')
    for Jahr in range (0,39):
        t=Jahr*365+59
        sch=int((Jahr)/4)   #Schaltjahr
        anfang=t+sch+tp_Zusatz#+31+30+31
        ende=anfang+10#+31+30+31
        gd=gdf.tp[anfang]
        for k in range (anfang,ende):
            gd=gd+gdf.tp[k]
        gdd=durschn(gd*ged.Masken[ma],ged.Masken[ma])
        print(float(gdd))
print('Until the end of the line')

In [ ]:
#einzelne Tage, Länder
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']
lis=[0,0,0,0,31,61]
start = time.time()

gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')
if len(gdf.time) > 14750 and len(gdf.time) < 15000:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(687,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
    
for ma in range (4,7): #(0,12)
    end=time.time()
    print('Land:',liste[ma],'              Zeit:',end-start)
    for Jahr in range (0,39):
        t=Jahr*365+59
        sch=int((Jahr+4)/4)   #Schaltjahr
        anfang=t+sch
        ende=anfang+31+30+31
        for k in range (anfang,ende):
            if ende-k > 61:
                Monat=3
            elif ende-k > 31 and ende-k < 62:
                Monat=4
            else:
                Monat=5
            Tag=-lis[Monat]+k-anfang+1
            gdd=durschn(gdf.tp[k]*ged.Masken[ma],ged.Masken[ma])
            if ma==0:
                if float(gdd) < 0:
                    print(Jahr+1980,Monat,Tag,0)
                else:
                    print(Jahr+1980,Monat,Tag,float(gdd))
            else:
                if float(gdd) < 0:
                    print(0)
                else:
                    print(float(gdd))
print('Until the end of the line')

In [8]:
#einzelne Tage, alles
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']
lis=[0,0,0,0,31,61]
start = time.time()

gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')
if len(gdf.time) > 14750 and len(gdf.time) < 15000:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(687,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
    

for Jahr in range (0,39):
    t=Jahr*365+59
    sch=int((Jahr+4)/4)   #Schaltjahr
    anfang=t+sch
    ende=anfang+31+30+31
    for k in range (anfang,ende):
        if ende-k > 61:
            Monat=3
        elif ende-k > 31 and ende-k < 62:
            Monat=4
        else:
            Monat=5
        Tag=-lis[Monat]+k-anfang+1
        gdd=np.sum(lii*gdf.tp[k])/(88*100)
        print(float(gdd))
print('Until the end of the line')

2.8257927367403366
3.267631567355498
3.153244814161823
3.053369925466636
2.7737278403904506
2.627908491679558
2.072078915691925
1.7355849756748447
0.9884834017735427
1.3510202518478718
1.2318081810008479
2.931443932964722
3.6883374512580627
2.169331271686848
0.9800746674174692
0.5099230167074674
0.2815311575267093
0.2702826702062319
0.6074176282970217
1.353365043020993
1.107922143988516
1.2570212621062689
1.1874006797785945
1.352644474769476
1.8349524150722978
2.2837932034017614
2.547579411638903
2.5816273372870904
2.7869600124707494
4.349459299404826
2.2036732815232085
1.6834228386280383
1.760749674255835
1.6686257513621565
1.799063576697739
1.6480116388780608
3.0202734451886184
4.102919711003971
4.132562650894938
4.851880223737993
3.4492691118996053
3.2190360058968275
4.574349631839171
6.717010900246176
6.679124991946748
5.219919041120363
4.644022816015668
5.045511482856383
3.9821096545359547
4.995588067871799
5.015324670412681
3.436670410529519
2.8052459206986606
2.0895207067514137


KeyboardInterrupt: 

In [ ]:
plt.imshow(lii*gdf.tp[120],cmap='turbo')

In [ ]:
(lii*gdf.tp[k])

In [ ]:
gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')
np.sum(gdf.tp[380])

In [ ]:
#Einzelner Monat
gdf=xr.open_dataset('/Users/weber-j/Documents/v750_all.nc')
gdf=gdf.isel(longitude=slice(0,88))
for Jahr in range (1,40): #(0,40)
    t=Jahr*365+59
    sch=int((Jahr+3)/4)   #Schaltjahr
    anfang=t+sch+31+30
    ende=anfang+31#+31+30+31
    gd=gdf.v[anfang]
    for k in range (anfang,ende):
        gd=gd+gdf.v[k]
        gdd=gd*lii
    print(float(np.mean(gdd))/(31))
    end=time.time()
print('Until the end of the line')

In [ ]:
#Ganzes Jahr, alles
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']

filename='/Users/weber-j/Documents/tp_all.nc'

gdf=xr.open_dataset(filename)
if filename == '/Users/weber-j/Documents/tp_all.nc':   #Regendaten sind kaputt (1979 doppelt drin), geht nur so
    gdf=gdf.isel(longitude=slice(0,88),time=slice(687,15000))
    tp_Zusatz=0
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))
    tp_Zusatz=1
for Jahr in range (0,39):
    t=Jahr*365
    sch=int((Jahr+3)/4)   #Schaltjahr
    anfang=t+sch
    if Jahr == 0 or Jahr == 4 or Jahr == 8 or Jahr == 12 or Jahr == 16 or Jahr == 20 or Jahr == 24 or Jahr == 28 or Jahr == 32 or
    Jahr == 36:
        ende=anfang+366
        scha=1
    else:
        ende=anfang+365
        scha=0
    gd=gdf.tp[anfang]
    for k in range (anfang,ende):
        gd=gd+gdf.tp[k]
        gdd=gd*lii
    print(float(np.mean(gdd)))
print('Until the end of the line')

In [ ]:
#Jährlich Länder
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']
gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')

if len(gdf.time) < 15000 and len(gdf.time) > 14900:   #Regendaten sind kaputt (1979 doppelt drin), geht nur so
    gdf=gdf.isel(longitude=slice(0,88),time=slice(687,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))

for ma in range (0,12): #(0,12)
    print(liste[ma],'   ########################################################')
    for Jahr in range (0,39): #(0,40)
        t=Jahr*365
        sch=int((Jahr+3)/4)   #Schaltjahr
        anfang=t+sch
        if Jahr == 0 or Jahr == 4 or Jahr == 8 or Jahr == 12 or Jahr == 16 or Jahr == 20 or Jahr == 24 or Jahr == 28 or Jahr == 32 
        or Jahr == 36:
            ende=anfang+366
        else:
            ende=anfang+365
        gd=gdf.tp[anfang]
        for k in range (anfang+1,ende):
            gd=gd+gdf.tp[k]
        gdd=durschn(gd*ged.Masken[ma],ged.Masken[ma])
        print(float(gdd))
print('Until the end of the line')

In [15]:
#Jährlich Länder auf Monatsbasis
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')
liste=['Burundi','Dschibuti','Eritrea','Äthiopien','Kenia','Ruanda','Somalia','Tansania','Südsudan','Sudan','Uganda','Indischer Ozean']
Month=[0,31,28,31,30,31,30,31,31,30,31,30,31]
Month_schalt=[0,31,29,31,30,31,30,31,31,30,31,30,31]
gdf=xr.open_dataset('/Users/weber-j/Documents/tp_all.nc')

if len(gdf.time) < 15000 and len(gdf.time) > 14900:   #Regendaten sind kaputt (1979 doppelt drin), geht nur so
    gdf=gdf.isel(longitude=slice(0,88),time=slice(687,15000))
else:
    gdf=gdf.isel(longitude=slice(0,88),time=slice(365,15000))

for ma in range (8,10): #(0,12)
    print(liste[ma],'   ########################################################')
    for Mon in range (1,13):
        for Jahr in range (0,39): #(0,39)
            t=Jahr*365
            sch=int((Jahr+3)/4)   #Schaltjahr
            if Jahr == 0 or Jahr == 4 or Jahr == 8 or Jahr == 12 or Jahr == 16 or Jahr == 20 or Jahr == 24 or Jahr == 28 or Jahr ==
            32 or Jahr == 36:
                anfang=t+sch+np.sum(Month_schalt[0:Mon])
                ende=anfang+Month_schalt[Mon]
            else:
                anfang=t+sch+np.sum(Month[0:Mon])
                ende=anfang+Month[Mon]
            if Jahr == 0:
                gd=gdf.tp[anfang]
            for k in range (anfang+1,ende):
                gd=gd+gdf.tp[k]
                #print(gdf.time[k])
        gd=gd/39
        gdd=durschn(gd*ged.Masken[ma],ged.Masken[ma])
        print(float(gdd))
print('Until the end of the line')

Südsudan    ########################################################
4.464586391558707
6.864279837141281
35.76620828878453
78.02712050612963
118.75870274240242
124.69457290582761
158.75131573001974
161.34875420603552
126.42123956520287
130.73139904273768
53.26838947582646
10.699382352637363
Sudan    ########################################################
0.0720720537981267
0.051551654801629966
2.7124821404644597
8.259530905973637
44.56302538794709
62.55587558047661
109.79271185572834
121.26801186567414
87.09437273733447
47.19677388804132
2.190559319487386
0.03911275882629268
Until the end of the line


In [ ]:
gdf=xr.open_dataset('/Users/weber-j/Documents/sp_all.nc')
gd=gdf.sp[anfang]

In [ ]:
for i in range (0,121):
    DeuCor.append(Deucor[i])

In [ ]:
gd=gd.isel(longitude=slice(0,88))

In [ ]:
gd*lii

In [ ]:
len(lii[0])

In [ ]:
for ma in range(0,12):
    print(float(durschn(gdf.sp[k]*ged.Masken[ma],ged.Masken[ma]))/100,liste[ma])

In [ ]:
gdf.sp[k]/100

In [ ]:
str(gd.time)

In [ ]:
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA.nc')
gdf=xr.open_dataset('/Users/weber-j/Documents/ws10_all.nc')

In [ ]:
ws=gdf.ws10[0]*ged.Masken[11]

In [ ]:
plt.imshow(ws)

In [ ]:
cnt=0
for n in range (0,100):
    for t in range (0,121):
        if float(ged.Masken[ma,n,t]) > 0:
            cnt=cnt+1
cnt

In [ ]:
plt.imshow(gd/9200,vmin=1000)
plt.colorbar()

In [ ]:
#Durchschnittliche Windrichtung an einem Ort (siehe /Users/weber-j/Documents/wr750_all.nc und /Users/weber-j/Documents/wr10_all.nc)
gdf=xr.open_dataset('/Users/weber-j/Documents/u750_all.nc')
ggf=xr.open_dataset('/Users/weber-j/Documents/v750_all.nc')
gd=180+180/np.pi*np.arctan2(np.asarray(ggf.v),np.asarray(gdf.u))
gf=(gdf.u*0+1)*gd
np.mean(gf[0:100000,0,52,22])

In [ ]:
#Monatliche Windrichtung an einem Punkt
'''Wind direction increases clockwise such that a northerly wind is 0°, an easterly wind is 90°,
a southerly wind is 180°, and a westerly wind is 270°.'''

gdf1=xr.open_dataset('/Users/weber-j/Documents/u750_all.nc')
ggf=xr.open_dataset('/Users/weber-j/Documents/v750_all.nc')
if len(gdf1.time) > 14750 and len(gdf1.time) < 15000:
    gdf1=gdf1.isel(longitude=slice(0,88),time=slice(687,15000))
    ggf=ggf.isel(longitude=slice(0,88),time=slice(687,15000))
else:
    gdf1=gdf1.isel(longitude=slice(0,88),time=slice(365,15000))
    ggf=ggf.isel(longitude=slice(0,88),time=slice(365,15000))

gdf1=gdf1.u[0:1000000,0,48,32]
ggf=ggf.v[0:1000000,0,48,32]
gdf=180+180/np.pi*np.arctan2(np.asarray(ggf),np.asarray(gdf1))

Tag=[31,29,31,30,31,30,31,31,30,31,30,31]
Tage=[31,28,31,30,31,30,31,31,30,31,30,31]

for Monat in range (0,12):
    ju=0
    for Jahr in range (0,39): #(0,40)
        t=Jahr*365
        sch=int((Jahr+3)/4)   #Schaltjahr
        if Jahr == 0 or Jahr == 4 or Jahr == 8 or Jahr == 12 or Jahr == 16 or Jahr == 20 or Jahr == 24 or Jahr == 28 or Jahr == 32 or Jahr == 36:
            anfang=t+sch+np.sum(Tag[0:Monat+1])-Tag[Monat]
            ende=anfang+Tag[Monat]
            gd=gdf[anfang]
            for k in range (anfang+1,ende):
                gd=gd+gdf[anfang]
            ju=ju+float(gd)/Tag[Monat]
        else:
            anfang=t+sch+np.sum(Tage[0:Monat+1])-Tage[Monat]
            ende=anfang+Tage[Monat]
            gd=gdf[anfang]
            for k in range (anfang+1,ende):
                gd=gd+gdf[anfang]
            ju=ju+float(gd)/Tage[Monat]
    ju=ju/39
    al=ju
    globals()['t2m'+str(Monat)]=al
Datas1=[t2m0,t2m1,t2m2,t2m3,t2m4,t2m5,t2m6,t2m7,t2m8,t2m9,t2m10,t2m11]
print('End')

In [ ]:
Datas1

In [4]:
gd=xr.open_dataset('/Users/weber-j/Documents/t2min_all.nc')

In [5]:
gd

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 721, longitude: 1440, time: 1341)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T11:30:00 ... 1982-09-02T11:30:00
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    t2min      (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Tue Sep 28 09:40:42 2021: cdo mergetime /pd/home/weber-j/Zw...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...

In [22]:
gd=xr.open_dataset('/Users/weber-j/Documents/t2min_all.nc')
gd=gd.isel(latitude=slice(303,403),longitude=slice(120,208))
gd

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 100, longitude: 88, time: 1341)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01T11:30:00 ... 1982-09-02T11:30:00
  * longitude  (longitude) float32 30.0 30.25 30.5 30.75 ... 51.25 51.5 51.75
  * latitude   (latitude) float32 14.25 14.0 13.75 13.5 ... -10.0 -10.25 -10.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    t2min      (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Tue Sep 28 09:40:42 2021: cdo mergetime /pd/home/weber-j/Zw...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...

In [36]:
#T2max/T2min Schnitt pro Land berechnen
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')

for ma in range (0,12):
    la=[]
    for yr in range (1987,1989):
        gd=xr.open_dataset('/Users/weber-j/Documents/ERA5/daily/ERA5_daily_t2max_'+str(yr)+'.nc')
        gd=gd.isel(latitude=slice(303,403),longitude=slice(120,208))
        if yr == 1980 or yr == 1984 or yr == 1988 or yr == 1992 or yr == 1996 or yr == 2000 or yr == 2004 or yr == 2008 or yr == 
        2012 or yr == 2016:
            start=60
        else:
            start=59
        t2max=gd.t2max[start]
        for day in range (start+1,start+92):
            t2max=t2max+gd.t2max[day]
        gdd=durschn(t2max*ged.Masken[ma],ged.Masken[ma])
        la.append(gdd/92)
    print(np.mean(la)-273.15)

20.208792742348237
28.177233119735718
29.033472993955286
24.49234168491637
25.908542623186577
20.45520805737476
27.35123802099963
22.577787005858
29.45475799271469
31.250463290299308
23.15263740684088
27.393244388368373


In [35]:
#T2max/T2min ausrechnen für die gesamte Area
ged=xr.open_dataset('/Users/weber-j/Documents/Masken/GHA_klein.nc')

la=[]
for yr in range (1987,1989):
    gd=xr.open_dataset('/Users/weber-j/Documents/ERA5/daily/ERA5_daily_t2m_'+str(yr)+'.nc')
    gd=gd.isel(latitude=slice(303,403),longitude=slice(120,208))
    if yr == 1980 or yr == 1984 or yr == 1988 or yr == 1992 or yr == 1996 or yr == 2000 or yr == 2004 or yr == 2008 or yr == 2012 
    or yr == 2016:
        start=60
    else:
        start=59
    t2max=gd.t2m[start]
    for day in range (start+1,start+92):
        t2max=t2max+gd.t2m[day]
    gdd=durschn(t2max*lii,lii)
    la.append(gdd/92)
print(np.mean(la)-273.15)

26.118165912657787
